In [3]:
%%time
%matplotlib inline
%run Prosser_header.py {dataset_name} {df_list}


Toggle code

Wall time: 9 ms


In [4]:
dataset_folder = "C:\\Users\\Marios\\Documents\\GitHub\\Thomas Prosser project\\datasets\\"
output_folder = "C:\\Users\\Marios\\Documents\\GitHub\\Thomas Prosser project\\output\\"

In [5]:
CSES_int = pd.read_stata(dataset_folder + "CSES"+os.sep+"integrated 1-4"+os.sep+"cses_imd.dta")

def get_dta_meta(fn):
    reader = pd.io.stata.StataReader( fn )
    header = reader.variable_labels()
    labels = reader.value_labels()
    return header,labels

# pd.read_stata(dataset_folder + "CSES"+os.sep+"module 5"+os.sep+"cses5.dta")
header5,labels5 = get_dta_meta(dataset_folder + "CSES"+os.sep+"module 5"+os.sep+"cses5.dta")
header4,labels4 = get_dta_meta(dataset_folder + "CSES"+os.sep+"module 4"+os.sep+"cses4.dta")
header3,labels3 = get_dta_meta(dataset_folder + "CSES"+os.sep+"module 3"+os.sep+"cses3.dta")
header2,labels2 = get_dta_meta(dataset_folder + "CSES"+os.sep+"module 2"+os.sep+"cses2.dta")
header1,labels1 = get_dta_meta(dataset_folder + "CSES"+os.sep+"module 1"+os.sep+"cses1.dta")
headerINT,labelsINT = get_dta_meta(dataset_folder + "CSES"+os.sep+"integrated 1-4"+os.sep+"cses_imd.dta")

In [6]:
### WEIGHTS

#  | Sample weights include those intended to correct for unequal
#  | selection probabilities resulting from "booster" samples,
#  | procedures for selection within the household, non-response,
#  | as well as other features of the sample design.
#  |
#  | Demographic weights adjust sample distributions of socio-
#  | demographic characteristics to more closely resemble the
#  | characteristics of the population.
#  |
#  | Political weights reconcile discrepancies in the reported
#  | electoral behavior of the survey respondents from the
#  | official vote counts.

# they don't advise combining weights (I'm not 100% clear why)

# but you will need to standardise the weights
# - within election studies (already done for IMD weights)
# - between election studies to equalise their effective sample size

sample_size_per_ES = CSES_int["IMD1010_1"].groupby(CSES_int["IMD1003"]).sum()
sample_weights_ssadj = CSES_int["IMD1010_1"].values/sample_size_per_ES.loc[CSES_int["IMD1003"]].values

sample_size_per_ES = CSES_int["IMD1010_2"].groupby(CSES_int["IMD1003"]).sum()
demographic_weights_ssadj = CSES_int["IMD1010_2"].values/sample_size_per_ES.loc[CSES_int["IMD1003"]].values

sample_size_per_ES = CSES_int["IMD1010_3"].groupby(CSES_int["IMD1003"]).sum()
political_weights_ssadj = CSES_int["IMD1010_3"].values/sample_size_per_ES.loc[CSES_int["IMD1003"]].values

In [8]:
df = pd.DataFrame()

In [9]:
### HOUSEHOLD INCOME

imd_var = "IMD2006"

# CSES_int[imd_var].cat.categories
# check to see if ordinal/categorical


non_response_cats = ['6. [SEE ELECTION STUDY NOTES]', '7. VOLUNTEERED: REFUSED', "8. VOLUNTEERED: DON'T KNOW", '9. MISSING']

df[imd_var+"_nr"] = CSES_int[imd_var].apply(lambda x: x if x in non_response_cats else np.nan)
top_cat = CSES_int[imd_var].cat.remove_categories(non_response_cats).cat.categories[-1]
df[imd_var+"__"+top_cat] = CSES_int[imd_var].cat.remove_categories(non_response_cats).cat.codes.replace(-1,np.nan)

In [187]:
## AGE: Categorical

# 4000 non-responses

# categorical -> dummies

imd_var = "IMD2001_2"

non_response_cats = ['9997. REFUSED', "9998. DON'T KNOW",'9999. MISSING']
replace_dict = dict(zip(non_response_cats,[np.nan]*len(non_response_cats)))

df[imd_var+"_nr"] = CSES_int[imd_var].apply(lambda x: x if x in non_response_cats else np.nan)
temp = pd.get_dummies(CSES_int[imd_var].cat.remove_categories(non_response_cats),prefix=imd_var)
df[temp.columns] = temp


In [ ]:
## AGE: Numerical

imd_var = "IMD2001_1"
# 6000 non-responses

non_response_cats = ['9997. REFUSED', "9998. DON'T KNOW",'9999. MISSING']
replace_dict = dict(zip(non_response_cats,[np.nan]*len(non_response_cats)))

df[imd_var+"_nr"] = CSES_int[imd_var].apply(lambda x: x if x in non_response_cats else np.nan)
top_cat = CSES_int[imd_var].cat.remove_categories(non_response_cats).cat.categories[-1]
df[imd_var+"__"+str(top_cat)] = CSES_int[imd_var].replace(replace_dict).astype(float)

In [191]:
## Gender

imd_var = "IMD2002"
CSES_int[imd_var].cat.categories

non_response_cats = ['3. [SEE ELECTION STUDY NOTES]', '7. REFUSED', '9. MISSING']
replace_dict = dict(zip(non_response_cats,[np.nan]*len(non_response_cats)))

## could work this different for two-valued variables ...

df[imd_var+"_nr"] = CSES_int[imd_var].apply(lambda x: x if x in non_response_cats else np.nan)
temp = pd.get_dummies(CSES_int[imd_var].cat.remove_categories(non_response_cats),prefix=imd_var)
df[temp.columns] = temp

In [194]:
## EDUCATION

# Reasonable question whether this is ordinal/categorical

imd_var = "IMD2003"
CSES_int[imd_var].cat.categories

non_response_cats = ['6. OTHER', '7. VOLUNTEERED: REFUSED', "8. VOLUNTEERED: DON'T KNOW", '9. MISSING']

replace_dict = dict(zip(non_response_cats,[np.nan]*len(non_response_cats)))

df[imd_var+"_nr"] = CSES_int[imd_var].apply(lambda x: x if x in non_response_cats else np.nan)
temp = pd.get_dummies(CSES_int[imd_var].cat.remove_categories(non_response_cats),prefix=imd_var)
df[temp.columns] = temp

In [ ]:
    IMD2001_1    >>>         AGE OF RESPONDENT (IN YEARS)
    IMD2001_2    >>>         AGE OF RESPONDENT (IN CATEGORIES)
    IMD2002      >>>         GENDER
    IMD2003      >>>         EDUCATION
    IMD2004      >>>         MARITAL STATUS
    IMD2005      >>>         RELIGIOUS DENOMINATION 
    IMD2006      >>>         HOUSEHOLD INCOME
    IMD2007      >>>         RURAL OR URBAN RESIDENCE

In [196]:
## MARITAL STATUS

imd_var = "IMD2004"
CSES_int[imd_var].cat.categories

non_response_cats = ['5. [SEE ELECTION STUDY NOTES]','7. VOLUNTEERED: REFUSED', "8. VOLUNTEERED: DON'T KNOW", '9. MISSING']

replace_dict = dict(zip(non_response_cats,[np.nan]*len(non_response_cats)))

df[imd_var+"_nr"] = CSES_int[imd_var].apply(lambda x: x if x in non_response_cats else np.nan)
temp = pd.get_dummies(CSES_int[imd_var].cat.remove_categories(non_response_cats),prefix=imd_var)
df[temp.columns] = temp

In [198]:
## RELIGIOUS DENOMINATION 

imd_var = "IMD2005"
CSES_int[imd_var].cat.categories

non_response_cats = ['97. VOLUNTEERED: REFUSED', "98. VOLUNTEERED: DON'T KNOW",'99. MISSING']

replace_dict = dict(zip(non_response_cats,[np.nan]*len(non_response_cats)))

df[imd_var+"_nr"] = CSES_int[imd_var].apply(lambda x: x if x in non_response_cats else np.nan)
temp = pd.get_dummies(CSES_int[imd_var].cat.remove_categories(non_response_cats),prefix=imd_var)
df[temp.columns] = temp

In [202]:
## RURAL OR URBAN

imd_var = "IMD2007"
CSES_int[imd_var].cat.categories

non_response_cats = [ '7. VOLUNTEERED: REFUSED', "8. VOLUNTEERED: DON'T KNOW", '9. MISSING']

replace_dict = dict(zip(non_response_cats,[np.nan]*len(non_response_cats)))

df[imd_var+"_nr"] = CSES_int[imd_var].apply(lambda x: x if x in non_response_cats else np.nan)
temp = pd.get_dummies(CSES_int[imd_var].cat.remove_categories(non_response_cats),prefix=imd_var)
df[temp.columns] = temp

In [30]:
cses_ids_to_pf = pd.read_csv(dataset_folder + "CSES"+os.sep+"Integration"+os.sep+"cses_ids_to_pf.csv")
cses_ids_to_pf = cses_ids_to_pf.set_index("Unnamed: 0")["0"]
cses_ids_to_pf = cses_ids_to_pf.dropna()

In [12]:
CSES_int["IMD3005_3"].value_counts()
# 9999999. MISSING                                           120168
# 9999997. VOLUNTEERED: REFUSED                                       1840
# 9999992. OTHER CANDIDATE/PARTY (NOT FURTHER SPECIFIED)              1380
# 9999998. VOLUNTEERED: DON'T KNOW                                    1304
# 9999988. NONE OF THE CANDIDATES PARTIES                              517
# 9999990. OTHER LEFT WING CANDIDATE/PARTY                                   67
# 9999991. OTHER RIGHT WING CANDIDATE/PARTY                                  36
# 9999989. INDEPENDENT CANDIDATE                                             34



missing_dict = {9999999:np.nan,
                9999997:np.nan,
                9999992:np.nan,
                9999998:np.nan,
                9999988:np.nan,
                9999990:np.nan,
                9999991:np.nan,
                9999989:np.nan,
               }


party_ident = CSES_int["IMD3005_3"].apply(lambda x: int(x[0:7])).replace(missing_dict)


In [33]:
%%time
pf_party_ident = party_ident.apply(lambda x: cses_ids_to_pf.loc[x] if pd.notnull(x) and (x in cses_ids_to_pf.index) else np.nan)

Wall time: 8.4 s


In [35]:
pf_party_ident.value_counts()

1998.0    3547
432.0     3371
424.0     3293
1824.0    2775
1474.0    2773
          ... 
7028.0       1
7005.0       1
7004.0       1
7024.0       1
4410.0       1
Name: IMD3005_3, Length: 628, dtype: int64

In [39]:
mask = df[imd_var+"__"+top_cat].notnull() & pf_party_ident.notnull()
mask.sum()

128831

In [38]:
demographic_weights_ssadj.shape

(281083,)

In [44]:
party_id_cses = CSES_int["IMD3005_3"][mask].cat.remove_unused_categories()

In [46]:
df[imd_var+"__"+top_cat][mask].groupby(party_id_cses[mask]).mean().sort_values()

IMD3005_3
6080080. PHL - Abang Lingkod, Inc.                       0.0
6080038. PHL - Anakpawis                                 0.0
6040033. PER - Project Country                           0.0
7640008. THA - Motherland Party (Phak Matunhum Party)    0.5
4840016. MEX - Cardenista Party                          0.5
                                                        ... 
6080052. PHL - Cooperative NATCCO Party                  4.0
0080031. ALB - Agrarian Party                            4.0
0760012. BRA - Solidarity                                4.0
7100007. ZAF - Freedom Front Plus                        4.0
0560002. BEL - Socialist Party (Francophone)             4.0
Name: IMD2006__5. HIGHEST HOUSEHOLD INCOME QUINTILE, Length: 634, dtype: float64

In [48]:
sample_size_mask = df[imd_var+"__"+top_cat][mask].groupby(party_id_cses[mask]).count()>100

In [50]:
sample_size_mask.sum()

225

In [54]:
df[imd_var+"__"+top_cat][mask].groupby(party_id_cses[mask]).mean()[sample_size_mask].sort_values().tail(50)
# 0-4

IMD3005_3
2030005. CZE - Green Party                                                 2.233083
7050001. SVN - Slovenian Democratic Party                                  2.251656
2080006. DNK - Danish Social Liberal Party                                 2.254717
7920002. TUR - Republican People's Party                                   2.256338
0560004. BEL - Flemish Liberals and Democrats                              2.268116
2080007. DNK - Conservative People's Party                                 2.272727
3520001. ISL - Independence Party                                          2.285135
2760006. DEU - Free Democratic Party                                       2.295322
1580003. TWN - People First Party                                          2.296392
3760004. ISR - Energy                                                      2.309211
7520003. SWE - Liberal People's Party                                      2.321070
1240006. CAN - Progressive Conservative                           

In [53]:
# BUG!
# 0760002. BRA - Brazilian Social Dmeocratic Party                           2.420755

4.0

In [ ]:
IMD3002_PR_1 >>>         CURRENT PRESIDENTIAL ELECTION: VOTE CHOICE - ROUND 1
IMD3002_PR_2 >>>         CURRENT PRESIDENTIAL ELECTION: VOTE CHOICE - ROUND 2
IMD3002_LH_PL >>>        CURRENT LOWER HOUSE ELECTION: VOTE CHOICE - PARTY LIST
IMD3002_LH_DC >>>        CURRENT LOWER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE
IMD3002_UH_PL >>>        CURRENT UPPER HOUSE ELECTION: VOTE CHOICE - PARTY LIST
IMD3002_UH_DC_1 >>>      CURRENT UPPER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE 1
IMD3002_UH_DC_2 >>>      CURRENT UPPER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE 2
IMD3002_UH_DC_3 >>>      CURRENT UPPER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE 3
IMD3002_UH_DC_4 >>>      CURRENT UPPER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE 4
IMD3002_OUTGOV  >>>      CURRENT MAIN ELECTION: VOTE CHOICE - OUTGOING GOVERNMENT (INCUMBENT)
    
IMD3004_PR_1 >>>         PREVIOUS PRESIDENTIAL ELECTION: VOTE CHOICE - ROUND 1
IMD3004_PR_2 >>>         PREVIOUS PRESIDENTIAL ELECTION: VOTE CHOICE - ROUND 2
IMD3004_LH_PL >>>        PREVIOUS LOWER HOUSE ELECTION: VOTE CHOICE - PARTY LIST
IMD3004_LH_DC >>>        PREVIOUS LOWER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE
IMD3004_UH_PL >>>        PREVIOUS UPPER HOUSE ELECTION: VOTE CHOICE - PARTY LIST
IMD3004_UH_DC_1 >>>      PREVIOUS UPPER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE 1
IMD3004_UH_DC_2 >>>      PREVIOUS UPPER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE 2
IMD3004_UH_DC_3 >>>      PREVIOUS UPPER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE 3

In [64]:
cses_vote_vars = ["IMD3002_PR_1",
"IMD3002_PR_2",
"IMD3002_LH_PL",
"IMD3002_LH_DC",
"IMD3002_UH_PL",
"IMD3002_UH_DC_1",
"IMD3002_UH_DC_2",
"IMD3002_UH_DC_3",
"IMD3002_UH_DC_4",
"IMD3004_PR_1",
"IMD3004_PR_2",
"IMD3004_LH_PL",
"IMD3004_LH_DC",
"IMD3004_UH_PL",
"IMD3004_UH_DC_1",
"IMD3004_UH_DC_2",
"IMD3004_UH_DC_3",]

In [83]:
missing_values = ['9999988. NONE OF THE CANDIDATES/PARTIES',
       '9999989. INDEPENDENT CANDIDATE',
       '9999990. OTHER LEFT WING CANDIDATE/PARTY',
       '9999991. OTHER RIGHT WING CANDIDATE/PARTY',
       '9999992. OTHER CANDIDATE/PARTY (NOT FURTHER SPECIFIED)',
       '9999993. INVALID/BLANK BALLOT',
       '9999995. NOT APPLICABLE: NO DISTRICT CANDIDATE VOTE',
       '9999995. NOT APPLICABLE: NO ROLE OF PRESIDENT',
       '9999995. NOT APPLICABLE: NOT A LIST SYSTEM',
       '9999995. NOT APPLICABLE: UNICAMERAL SYSTEM',
       '9999996. NOT APPLICABLE: NO LOWER HOUSE ELECTION',
       '9999996. NOT APPLICABLE: NO PRESIDENTIAL ELECTION',
       '9999996. NOT APPLICABLE: NO PRESIDENTIAL ELECTION / NO SECOND ROUND',
       '9999996. NOT APPLICABLE: NO UPPER HOUSE ELECTION OR CANDIDATE VOTE',
       '9999996. NOT APPLICABLE: NO UPPER HOUSE ELECTION OR LIST VOTE',
       '9999997. VOLUNTEERED: REFUSED', "9999998. VOLUNTEERED: DON'T KNOW",
       '9999999. MISSING/ABSTAINED (DID NOT VOTE)']

In [66]:
stacked_vote_vars = CSES_int[cses_vote_vars].stack()

In [72]:
stacked_vote_vars.value_counts().sort_index().tail(20).index

Index(['8580005. URY - Popular Assembly', '8580006. URY - Civic Union',
       '9999988. NONE OF THE CANDIDATES/PARTIES',
       '9999989. INDEPENDENT CANDIDATE',
       '9999990. OTHER LEFT WING CANDIDATE/PARTY',
       '9999991. OTHER RIGHT WING CANDIDATE/PARTY',
       '9999992. OTHER CANDIDATE/PARTY (NOT FURTHER SPECIFIED)',
       '9999993. INVALID/BLANK BALLOT',
       '9999995. NOT APPLICABLE: NO DISTRICT CANDIDATE VOTE',
       '9999995. NOT APPLICABLE: NO ROLE OF PRESIDENT',
       '9999995. NOT APPLICABLE: NOT A LIST SYSTEM',
       '9999995. NOT APPLICABLE: UNICAMERAL SYSTEM',
       '9999996. NOT APPLICABLE: NO LOWER HOUSE ELECTION',
       '9999996. NOT APPLICABLE: NO PRESIDENTIAL ELECTION',
       '9999996. NOT APPLICABLE: NO PRESIDENTIAL ELECTION / NO SECOND ROUND',
       '9999996. NOT APPLICABLE: NO UPPER HOUSE ELECTION OR CANDIDATE VOTE',
       '9999996. NOT APPLICABLE: NO UPPER HOUSE ELECTION OR LIST VOTE',
       '9999997. VOLUNTEERED: REFUSED', '9999998. VOLUNTEER

In [76]:
CSES_int[cses_vote_vars].replace( dict(zip(missing_values,[np.nan]*len(missing_values)))).notnull().sum().sort_values()

IMD3004_UH_DC_3        58
IMD3002_UH_DC_4       176
IMD3002_UH_DC_3       609
IMD3004_UH_DC_2       822
IMD3004_UH_PL        1843
IMD3002_UH_DC_2      3873
IMD3004_UH_DC_1      5700
IMD3002_UH_PL        8989
IMD3004_PR_2        11766
IMD3002_PR_2        16430
IMD3002_UH_DC_1     25164
IMD3004_PR_1        38302
IMD3004_LH_DC       48732
IMD3002_PR_1        50059
IMD3002_LH_DC       80302
IMD3004_LH_PL       82896
IMD3002_LH_PL      132594
dtype: int64

In [82]:
# party_id_cses = CSES_int["IMD3005_3"][mask].cat.remove_unused_categories()

In [ ]:
## Same quick breakdown, but using a different vote variable
# IMD3002_LH_PL     highest num non-null values!

In [91]:
lower_house_party_list = CSES_int["IMD3002_LH_PL"][mask].replace(dict(zip(missing_values,[np.nan]*len(missing_values))))
lower_house_party_list.value_counts()

mask = df[imd_var+"__"+top_cat].notnull() & lower_house_party_list.notnull()

sample_size_mask = df[imd_var+"__"+top_cat][mask].groupby(lower_house_party_list[mask]).count()>100

In [93]:
df[imd_var+"__"+top_cat][mask].groupby(lower_house_party_list[mask]).mean()[sample_size_mask].sort_values().tail(10)
# 0-4

IMD3002_LH_PL
6160001. POL - Civic Platform                                   2.561028
7560021. CHE - Green Liberal Party                              2.638655
6040052. PER - National Unity                                   2.645161
5780002. NOR - Conservative Party                               2.654362
6160008. POL - Freedom Union                                    2.690476
2030010. CZE - TOP 09                                           2.719178
5540004. NZL - Act New Zealand                                  2.723247
2460003. FIN - National Coalition Party                         2.748503
1000001. BGR - Citizens for European Development of Bulgaria    2.796875
6040051. PER - Alliance for the Great Change                    2.818182
Name: IMD2006__5. HIGHEST HOUSEHOLD INCOME QUINTILE, dtype: float64

In [94]:
df[imd_var+"__"+top_cat][mask].groupby(lower_house_party_list[mask]).mean()[sample_size_mask].sort_values().head(10)
# 0-4

IMD3002_LH_PL
7640001. THA - For Thais Party                              1.058824
7100001. ZAF - African National Congress                    1.157343
2030004. CZE - Communist Party of Bohemia and Moravia       1.359909
6160024. POL - Self-Defense of the Republic Poland          1.567797
5540009. NZL - Alliance                                     1.622222
1520004. CHL - Socialist Party                              1.629310
5540003. NZL - New Zealand First                            1.629328
3920012. JPN - New Frontier Party                           1.636364
3000001. GRC - Pan-Hellenic Socialist Movement              1.643059
6430002. RUS - Communist Party of the Russian Federation    1.670554
Name: IMD2006__5. HIGHEST HOUSEHOLD INCOME QUINTILE, dtype: float64

In [ ]:
## do a version with weights added in, prob won't make much difference but just in case!

In [95]:
cses_variable_types = pd.read_pickle(dataset_folder + "CSES"+os.sep+"Integration"+os.sep+"cses_variable_types.pkl")
cses_variable_types[cses_variable_types["variable type"]=="party"]

CSES_int["IMD5000_A"]

0                  0080002. ALB - Democratic Party
1                  0080002. ALB - Democratic Party
2                  0080002. ALB - Democratic Party
3                  0080002. ALB - Democratic Party
4                  0080002. ALB - Democratic Party
                            ...                   
281078    7100001. ZAF - African National Congress
281079    7100001. ZAF - African National Congress
281080    7100001. ZAF - African National Congress
281081    7100001. ZAF - African National Congress
281082    7100001. ZAF - African National Congress
Name: IMD5000_A, Length: 281083, dtype: category
Categories (107, object): [0080002. ALB - Democratic Party < 0320001. ARG - Front for the Victory < 0360001. AUS - Liberal Party < 0360003. AUS - Australian Labor Party ... 8260002. GBR - Labour < 8400001. USA - Republican Party < 8400002. USA - Democratic Party < 8580001. URY - Broad Front]

In [102]:
cses_variable_types[cses_variable_types["variable type"]=="party"].head(50)

,description,variable type
IMD5000_A,PARTY A IDENTIFIER - NUMERICAL,party
IMD5000_B,PARTY B IDENTIFIER - NUMERICAL,party
IMD5000_C,PARTY C IDENTIFIER - NUMERICAL,party
IMD5000_D,PARTY D IDENTIFIER - NUMERICAL,party
IMD5000_E,PARTY E IDENTIFIER - NUMERICAL,party
IMD5000_F,PARTY F IDENTIFIER - NUMERICAL,party
IMD5000_G,PARTY G IDENTIFIER - NUMERICAL,party
IMD5000_H,PARTY H IDENTIFIER - NUMERICAL,party
IMD5000_I,PARTY I IDENTIFIER - NUMERICAL,party
IMD5001_A,PERCENT VOTE - LOWER HOUSE - PARTY A,party


In [99]:
CSES_int["IMD1008_YEAR"]

0         2005
1         2005
2         2005
3         2005
4         2005
          ... 
281078    2014
281079    2014
281080    2014
281081    2014
281082    2014
Name: IMD1008_YEAR, Length: 281083, dtype: int16

In [ ]:
## Year x Party


# party_A_identifier, year, party_A_left-right-pos



In [104]:
CSES_int[["IMD5000_A","IMD1008_YEAR","IMD5012_A",]]

,IMD5000_A,IMD1008_YEAR,IMD5012_A
0,0080002. ALB - Democratic Party,2005,6
1,0080002. ALB - Democratic Party,2005,6
2,0080002. ALB - Democratic Party,2005,6
3,0080002. ALB - Democratic Party,2005,6
4,0080002. ALB - Democratic Party,2005,6
...,...,...,...
281078,7100001. ZAF - African National Congress,2014,3
281079,7100001. ZAF - African National Congress,2014,3
281080,7100001. ZAF - African National Congress,2014,3
281081,7100001. ZAF - African National Congress,2014,3


In [110]:
["99. MISSING","97. NOT APPLICABLE"]

CSES_int["IMD5012_A"].value_counts().sort_index()

00. LEFT               2000
1.0                    4738
2.0                    4141
3.0                   44590
4.0                   44562
4.5                    2971
5.0                   36751
6.0                   37612
7.0                   62527
8.0                   16402
9.0                   11332
97. NOT APPLICABLE     1009
99. MISSING           12448
Name: IMD5012_A, dtype: int64

In [111]:
CSES_int["IMD5012_B"].value_counts().sort_index()

# dubious - check originals!

# Some collaborators have reported sub-incremental values (i.e.
#          | 4.5, 7.5, etc.).

00. LEFT               1496
1.0                    1000
2.0                    4641
3.0                   35066
4.0                   53598
5.0                   37917
5.5                    1798
6.0                   25859
7.0                   56581
8.0                   44295
9.0                    5375
97. NOT APPLICABLE     1009
99. MISSING           12448
Name: IMD5012_B, dtype: int64

In [154]:
### Left-Right Party_A

for party in ["A","B","C","D","E","F","G","H","I"]:

    imd_var = "IMD5012_"+party

    # CSES_int[imd_var].cat.categories
    # check to see if ordinal/categorical


    non_response_cats = ["99. MISSING","97. NOT APPLICABLE"]
    replace_dict = {"99. MISSING":np.nan,"97. NOT APPLICABLE":np.nan,'00. LEFT':0,'10. RIGHT':10}


    df[imd_var+"_nr"] = CSES_int[imd_var].apply(lambda x: x if x in non_response_cats else np.nan)
    top_cat = '10. RIGHT'
    df[imd_var+"__"+top_cat] = CSES_int[imd_var].replace(replace_dict).astype(float)

In [161]:
for party in ["A","B","C","D","E","F","G","H","I"]:

    imd_var = "IMD5012_"+party

    mask = df[imd_var+"__"+top_cat].notnull() & (CSES_int["IMD5000_"+party]!="9999999. MISSING")

    sample_size_mask = df[imd_var+"__"+top_cat][mask].groupby(CSES_int["IMD5000_"+party][mask]).count()>100

    display( df[imd_var+"__"+top_cat][mask].groupby(CSES_int["IMD5000_"+party][mask]).mean()[sample_size_mask].sort_values() )

IMD5000_A
1120001. BLR - Communist Party of Belarus                   0.000000
8040010. UKR - Communist Party of Ukraine                   1.000000
6430002. RUS - Communist Party of the Russian Federation    1.000000
3000003. GRC - Coalition of the Radical Left                2.000000
7050016. SVN - Zoran Jankovic List - Positive Slovenia      2.000000
                                                              ...   
7560001. CHE - Swiss People's Party                         8.398421
2030002. CZE - Civic Democratic Party                       8.619622
4990011. MNE - Coalition 'For a European Montenegro'        9.000000
6040001. PER - Popular Force                                9.000000
1520002. CHL - Independent Democratic Union                 9.000000
Name: IMD5012_A__10. RIGHT, Length: 101, dtype: float64

IMD5000_B
6430002. RUS - Communist Party of the Russian Federation    0.0
3800011. ITA - Democrats of the Left                        2.0
3000003. GRC - Coalition of the Radical Left                2.0
4840003. MEX - Democratic Revolution Party                  2.0
1120004. BLR - Agrarian Party                               2.0
                                                           ... 
1520002. CHL - Independent Democratic Union                 8.0
5780002. NOR - Conservative Party                           8.0
2030002. CZE - Civic Democratic Party                       9.0
6040001. PER - Popular Force                                9.0
2500007. FRA - National Front                               9.0
Name: IMD5012_B__10. RIGHT, Length: 109, dtype: float64

IMD5000_C
2030004. CZE - Communist Party of Bohemia and Moravia     0.716564
3000004. GRC - Communist Party of Greece                  1.000000
3920004. JPN - Japanese Communist Party                   1.000000
4990003. MNE - Socialist People's Party                   1.000000
6160017. POL - Palikots Movement                          1.000000
                                                           ...    
2460003. FIN - National Coalition Party                   9.000000
2500007. FRA - National Front                            10.000000
3000006. GRC - Golden Dawn                               10.000000
1120017. BLR - Christian Conservative Party              10.000000
2030010. CZE - TOP 09                                    10.000000
Name: IMD5012_C__10. RIGHT, Length: 121, dtype: float64

IMD5000_D
3920004. JPN - Japanese Communist Party             0.567458
0320004. ARG - Workers' Left Front                  1.000000
0360005. AUS - Australian Greens                    1.000000
2500006. FRA - Left Front                           1.000000
3520005. ISL - Left-Green Movement                  1.642011
                                                     ...    
2030010. CZE - TOP 09                               9.000000
5540004. NZL - Act New Zealand                      9.000000
3480004. HUN - Hungarian Justice and Life Party     9.000000
1120027. BLR - The BNF Party                        9.000000
2500007. FRA - National Front                      10.000000
Name: IMD5012_D__10. RIGHT, Length: 114, dtype: float64

IMD5000_E
8580005. URY - Popular Assembly                  0.0
2500009. FRA - Workers' Struggle                 0.0
3440026. HKG - April Fifth Action                1.0
4100015. KOR - Democratic Labor Party            1.0
2500004. FRA - French Communist Party            1.0
                                                ... 
3920007. JPN - Japan Restoration Party           9.0
7030012. SVK - People's Party - Our Slovakia    10.0
0560012. BEL - Flemish Block                    10.0
3000006. GRC - Golden Dawn                      10.0
7050008. SVN - Slovenian National Party         10.0
Name: IMD5012_E__10. RIGHT, Length: 115, dtype: float64

IMD5000_F
1520006. CHL - Communist Party of Chile                      0.0
6200007. PRT - Portugese Communist Worker's Party            0.0
3440006. HKG - Labour Party                                  1.0
3800013. ITA - Communist Refoundation Party                  1.0
6160007. POL - Labour Union                                  1.0
                                                            ... 
4280014. LVA - Latvian Association of the Regions            9.0
1120005. BLR - United Civil Party                            9.0
6160014. POL - League of Polish Families                     9.0
3760050. ISR - National Union - National Religious Party     9.0
7050008. SVN - Slovenian National Party                     10.0
Name: IMD5012_F__10. RIGHT, Length: 108, dtype: float64

IMD5000_G
2500014. FRA - Revolutionary Communist League           0.0
4840011. MEX - Possible Mexico                          1.0
3000004. GRC - Communist Party of Greece                1.0
5540011. NZL - MANA Movement                            1.0
6040007. PER - Direct Democracy                         1.0
                                                       ... 
2760009. DEU - National Democratic Party Of Germany     9.0
6160004. POL - Solidarity Electoral Action              9.0
7920015. TUR - Great Union Party                        9.0
6160018. POL - Congress of the New Right               10.0
1000007. BGR - Attack                                  10.0
Name: IMD5012_G__10. RIGHT, Length: 72, dtype: float64

IMD5000_H
0400006. AUT - Communist Party of Austria                                 1.000000
5780010. NOR - Red Electoral Alliance                                     1.000000
5780009. NOR - Red Party                                                  1.000000
2080005. DNK - United List - Red-Greens                                   1.000000
4840008. MEX - Social Democratic Party                                    1.000000
8400005. USA - Green Party                                                2.000000
6430005. RUS - Pensioners Party                                           2.000000
5540009. NZL - Alliance                                                   2.000000
3920006. JPN - Social Democratic Party                                    2.000000
3760004. ISR - Energy                                                     2.000000
6160068. POL - Social Democracy of Poland                                 3.000000
2760014. DEU - Pirates Party                                              3.0

IMD5000_I
4840004. MEX - Labor Party                            1.000000
5780009. NOR - Red Party                              1.000000
3760010. ISR - United Arab List                       1.000000
2500004. FRA - French Communist Party                 1.000000
6160007. POL - Labour Union                           1.000000
6200026. PRT - Humanist Party                         2.000000
7520009. SWE - Feminist Party                         2.000000
0560013. BEL - Confederated Ecologists                3.000000
3920005. JPN - Green Wind                             3.000000
6080009. PHL - People's Reform Party                  4.000000
4990007. MNE - Croatian Civic Initiative              4.000000
1910011. HRV - Istrian Democratic Assembly            4.000000
1520007. CHL - Radical Social-democratic Party        4.000000
1000009. BGR - Movement 21                            4.000000
7240012. ESP - Union, Progress and Democracy          4.000000
4840026. MEX - Mexican Liberal Party         

In [158]:
df[imd_var+"__"+top_cat][mask].groupby(CSES_int["IMD5000_A"][mask]).mean()[sample_size_mask].sort_values()

IMD5000_A
1120001. BLR - Communist Party of Belarus                   0.000000
8040010. UKR - Communist Party of Ukraine                   1.000000
6430002. RUS - Communist Party of the Russian Federation    1.000000
3000003. GRC - Coalition of the Radical Left                2.000000
7050016. SVN - Zoran Jankovic List - Positive Slovenia      2.000000
                                                              ...   
7560001. CHE - Swiss People's Party                         8.398421
2030002. CZE - Civic Democratic Party                       8.619622
4990011. MNE - Coalition 'For a European Montenegro'        9.000000
6040001. PER - Popular Force                                9.000000
1520002. CHL - Independent Democratic Union                 9.000000
Name: IMD5012_A__10. RIGHT, Length: 101, dtype: float64

9999999. MISSING                                        210169
5280007. NLD - Reformed Political Party                   6086
0760009. BRA - Democrats                                  3136
5540006. NZL - United Future New Zealand                  2598
0760046. BRA - Liberal Party                              2514
7520009. SWE - Feminist Party                             2379
6420020. ROU - Democratic Liberal Party                   2283
0560013. BEL - Confederated Ecologists                    2225
2080011. DNK - Centre Democrats                           2026
2500018. FRA - Radical Party of the Left                  2000
0760010. BRA - Brazilian Labor Party                      2000
4840026. MEX - Mexican Liberal Party                      1991
3920005. JPN - Green Wind                                 1937
8400006. USA - Constitution Party                         1929
2760009. DEU - National Democratic Party Of Germany       1889
6160007. POL - Labour Union                            